# Web crawling

El web crawling consiste en extraer los enlaces de una página web e ir recorriéndolos para descubrir el contenido de la web. Los enlaces pueden ser a otros sitios web o pertenecer a un mismo sitio web. En este ejercicio, vamos a implementar un crawler para extraer el contenido de la web https://books.toscrape.com/. Para ello, deberás completar los métodos de la clase WebCrawler.

Para trabajar con las URLs que encontremos en las páginas, usaremos la función "urljoin(url_base, url_nueva)" que lo que hace es ver si url_nueva es absoluta o relativa. Si es absoluta, devuelve esa misma URL, pero si es relativa, la concatena a la url_base.

In [1]:
# Primero, cargaremos los paquetes necesarios
from bs4 import BeautifulSoup as Soup
import os
import re
import pip._vendor.requests as requests
import csv
import pandas as pd
from urllib.parse import urljoin

In [2]:
class WebCrawler:
    def __init__(self, base_url, output_dir="output"):
        self.base_url = base_url
        self.output_dir = output_dir
        # usamos un diccionario porque queremos poder comprobar rápidamente si una URL ya se ha descargado y a la vez
        # poder recorrer el diccionario en el orden en el que hemos insertado los elementos (el diccionario de Python
        # mantiene el orden de inserción). Insertaremos las URLs como la clave, y pondremos '' como valor asociado.
        self.urls = {base_url: ""}
        
        # Crear el directorio de salida si no existe
        if not os.path.exists(self.output_dir):
            os.mkdir(self.output_dir)
    
    def sanitize_text(self, text):
        """Limpia el texto para eliminar caracteres no permitidos en nombres de archivo y convertir las mayúsculas en minúsculas
        Args:
            text : Texto que se quiere limpiar.
            
        Return:
            clean_text : Texto limpio.
        """
        
        clean_text = re.sub(r'\s+', ' ', text)  # sustituye cualquier secuencia de espacios y tabuladores por un espacio solamente
        clean_text = re.sub(r'[\\/*?:"<>|]', "-", clean_text)  # sustituye los caracteres \/*?:"<>| por un guión -
        clean_text = clean_text.lower()

        return clean_text

    def download_page(self, url):
        """Descarga el contenido HTML de una página
        Args:
            url : url de la web.
            
        Return:
            req.text : texto html de la página.
        """
        ## BEGIN YOUR CODE
        req = requests.get(url)
        ## END YOUR CODE
        return req.text

    def save_content(self, url, content):
        """Guarda el contenido de texto de la página en un archivo. ¡OJO! Solo el contenido de texto, no las etiquetas del HTML
        Args:
            url : url de la web.
            content: texto extraído del objeto BeautifulSoup con el contenido de la web.
        """
        ## BEGIN YOUR CODE
        # Crear nombre de archivo a partir de la URL
        filename = self.sanitize_text(url.replace('https://', "").replace('http://', "").replace("/", "-"))
        filepath = os.path.join(self.output_dir, filename)

        # Guardar solo el texto (sin etiquetas HTML)
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(content.get_text())
        ## END YOUR CODE

    def crawl(self):
        """Realiza el proceso de crawling en el sitio web"""

        i = 0
        while i < len(self.urls.keys()):

        ## BEGIN YOUR CODE
            current_url = list(self.urls.keys())[i]

            # Descargar la página
            html_content = self.download_page(current_url)

            # Parsear con BeautifulSoup
            soup = Soup(html_content, 'html.parser')

            # Guardar el contenido 
            self.save_content(current_url, soup)

            # Extraer enlaces de esta página
            self.extract_links(soup, current_url)
        ## END YOUR CODE

            i += 1

    def extract_links(self, soup, current_url):
        """Extrae enlaces de una página y los agrega a la lista de URLs (self.urls) si son nuevos
        Args:
            soup : objeto BeautifulSoup con el contenido de la web.
            current_url: url de la que se está extrayendo la información.
        """
        ## BEGIN YOUR CODE
        # Buscar todos los enlaces <a> con atributo 'href'
        links = soup.find_all('a', href=True)

        for link in links:
            # Obtener la URL del enlace
            href = link['href']

            # Convertir a URL absolita usando 'urljoin'
            absolute_url = urljoin(current_url, href)

            # Agregar solo si es del mismo dominio y no está ya en el diccionario
            if absolute_url.startswith(self.base_url) and absolute_url not in self.urls:
                self.urls[absolute_url] = ""
        ## END YOUR CODE

    def run(self):
        """Método principal para iniciar el proceso de crawling"""
        print("Iniciando crawling...")
        self.crawl()
        print("Crawling completado.")
        print(f"URLs encontradas: {len(self.urls.keys())}")

Ejecuta ahora el scraper para recorrer la web y descargar su contenido. OJO, puede tardar unos minutos, así que puedes ver el directorio "output" y comprobar si van descargándose las webs.

In [10]:
crawler = WebCrawler("https://books.toscrape.com/")
if not os.path.exists("output") or len(os.listdir("output")) == 0:
    crawler.run()
else:
    print("Output folder already exists, skipping crawling...")
print(crawler.urls)

Output folder already exists, skipping crawling...
{'https://books.toscrape.com/': ''}


Salida:
```
Iniciando crawling...
Crawling completado.
URLs encontradas: 1195
{'https://books.toscrape.com/': '', 'https://books.toscrape.com/index.html': '', 'https://books.toscrape.com/catalogue/category/books_1/index.html': '' ...
```

El total de URLs encontradas debe ser de 1195.

### Cerrando el círculo: Obtención del índice invertido del sitio web books.toscrape

Ahora que sabemos cómo se realiza el proceso de obtención de la información de documentos web, vamos a construir su índice invertido para poder buscar información. Vemos así cómo hemos conseguido implementar todos los procesos de un sistema de recuperación de información mediante cada una de las prácticas.

A continuación, se deberán leer los documentos para extraer el vocabulario y calcular la frecuencia de término según se realizó en la primera práctica de la asignatura. **Esto nos permitirá comprobar que hemos leido correctamente las páginas web**. Para ello, copia las celdas correspondientes y modifícalas según sea necesario. Al final muestra el número de términos que contiene la colección, el tamaño del vocabulario y los 10 términos más frecuentes con su conteo.

In [11]:
## BEGIN YOUR CODE
import glob
from collections import Counter

# Obtener lista de archivos descargados
file_paths = sorted(glob.glob(os.path.join("output", "*.html")))

# Diccionario para almacenar la frecuencia de términos
term_frequency = Counter()
total_terms = 0

# Leer cada documento
for filepath in file_paths:
    with open(filepath, 'r', encoding='utf-8') as f:
        content = f.read()

        # Tokenizar el texto (separar por espacios y convertir a minúsculas)
        tokens = content.lower().split()

        # Contar términos
        term_frequency.update(tokens)
        total_terms += len(tokens)

# Tamaño del vocabulario
vocabulary_size = len(term_frequency)

# Mostrar resultados
print(f"Número de términos en la colección: {total_terms}")
print(total_terms)
print(f"Tamaño del vocabulario: {vocabulary_size}")
print("Los 10 términos más frecuentes son:")

# Obtener los 10 términos más frecuentes
most_common = term_frequency.most_common(10)
for term, count in most_common:
    print(f"\t{term}: {count}")
## END YOUR CODE


Número de términos en la colección: 444828
444828
Tamaño del vocabulario: 35815
Los 10 términos más frecuentes son:
	to: 18351
	the: 18196
	in: 15878
	and: 12179
	stock: 11264
	of: 10936
	a: 9660
	add: 9539
	basket: 9284
	...: 6055


```
Número de términos en la colección: 444877

444877

Tamaño del vocabulario : 35462

Los 10 términos más frecuentes son:

  to: 18380

  the: 18220

  in: 15913

  and: 12193

  stock: 11284

  of: 10940

  a: 9668

  add: 9560

  basket: 9304
  
  ...: 6065
```

Una vez extraído el vocabulario y la frecuencia de término, construye ahora el índice invertido de los documentos copiando de la práctica 1 el código correspondiente y modificando lo que sea necesario. Vamos a usar el índice sencillo que hicimos al comienzo que simplemente es un diccionario de términos la posting list de documentos donde aparece cada término. Puedes usar todas las celdas que necesites.

In [12]:
## BEGIN YOUR CODE
from collections import defaultdict

# Crear el índice invertido como un diccionario donde:
#   - clave: término
#   - valor: lista de IDs de documentos donde aparece el término
inverted_index = defaultdict(list)

# Procesar cada documento
for doc_id, filepath in enumerate(file_paths):
    with open(filepath, 'r', encoding='utf-8') as f:
        content = f.read()

        # Tokenizar el texto (separar por espacios y convertir a minúsculas)
        tokens = content.lower().split()

        # Obtener términos únicos del documento
        unique_terms = set(tokens)

        # Añadir el doc_id a la posting list de cada término
        for term in unique_terms:
            inverted_index[term].append(doc_id)
## END YOUR CODE

In [6]:
sorted_inverted_index = dict(sorted(inverted_index.items()))  # suponemos que el índice invertido se llama inverted_index

print(sorted_inverted_index)

{'!': [285], '!!': [491], '"': [40, 130, 580, 628, 1113], '"...the': [868], '"12': [2], '"a': [30, 40, 45, 129, 311, 452, 506, 623, 727, 844, 1042, 1044], '"adventure"': [96], '"agnostic,"': [290], '"aladdin': [52], '"all': [67], '"all"': [719], '"amy': [61], '"an': [40, 301, 825], '"antony': [301], '"at': [566], '"atheist,"': [290], '"autotheory"': [837], '"baboushka"': [301], '"barrel': [444], '"basil\'s': [33], '"beat"': [623], '"behind': [312], '"bella?"edward\'s': [353], '"bestselling': [535], '"between': [756], '"beyond': [1016], '"big"â\x80\x94and': [607], '"blink":': [109], '"book': [556], '"boston': [126], '"brant': [40], '"brazen"': [122], '"break': [1028], '"bridget': [124], '"by': [1181], '"cat"': [395], '"catches"': [343], '"chair."mixed': [336], '"chicago': [40], '"children': [606], '"christine': [33], '"cleanse"': [417], '"codename': [301], '"comprised': [580], '"consistently': [7], '"content': [336], '"crumbled': [344], '"cynical': [868], '"dean': [623], '"deed,"': [603

Salida (¡OJO! es tan larga que se puede quedar colgado VSCode si la metemos entera en una celda de markdown):

```
{'!': [286], '!!': [492], '#01-#05': [1186], '#1': [1, 11, 21, 82, 85, 86, 97, 294, 297, 317, 319, 322, 323, 370, 373, 382, 385, 386, 520, 55
...
 '\ufeffintroduction': [1024], '\ufeffwritten': [1024]}

 ```




Buscamos ahora la lista de postings para el término "dagger" y obtenemos los nombres de documento correspondientes.

In [13]:
posting_list = inverted_index['dagger']  
posting_list

[171,
 209,
 210,
 258,
 260,
 284,
 316,
 425,
 429,
 431,
 432,
 463,
 498,
 568,
 668,
 670,
 692,
 794,
 853,
 1044,
 1176]

In [14]:
def retrieve_docnames(filenames, docids):
   
    return [filenames[i] for i in list(docids)]
    
    
retrieve_docnames(file_paths,posting_list)   # suponemos que file_paths contiene una lista con las rutas a los documentos ordenados según docid

['output/books.toscrape.com-catalogue-category-books-fantasy_19-page-2.html',
 'output/books.toscrape.com-catalogue-category-books-romance_8-index.html',
 'output/books.toscrape.com-catalogue-category-books-romance_8-page-1.html',
 'output/books.toscrape.com-catalogue-category-books_1-page-33.html',
 'output/books.toscrape.com-catalogue-category-books_1-page-35.html',
 'output/books.toscrape.com-catalogue-changing-the-game-play-by-play-2_317-index.html',
 'output/books.toscrape.com-catalogue-dark-lover-black-dagger-brotherhood-1_319-index.html',
 'output/books.toscrape.com-catalogue-grey-fifty-shades-4_592-index.html',
 'output/books.toscrape.com-catalogue-harry-potter-and-the-chamber-of-secrets-harry-potter-2_325-index.html',
 'output/books.toscrape.com-catalogue-harry-potter-and-the-half-blood-prince-harry-potter-6_326-index.html',
 'output/books.toscrape.com-catalogue-harry-potter-and-the-order-of-the-phoenix-harry-potter-5_327-index.html',
 'output/books.toscrape.com-catalogue-i-ha

```
['output\\https---books.toscrape.com-catalogue-category-books-fantasy_19-page-2.html',

 'output\\https---books.toscrape.com-catalogue-category-books-romance_8-index.html',

 'output\\https---books.toscrape.com-catalogue-category-books-romance_8-page-1.html',

 'output\\https---books.toscrape.com-catalogue-category-books_1-page-33.html',

 'output\\https---books.toscrape.com-catalogue-category-books_1-page-35.html',

 'output\\https---books.toscrape.com-catalogue-changing-the-game-play-by-play-2_317-index.html',

 'output\\https---books.toscrape.com-catalogue-dark-lover-black-dagger-brotherhood-1_319-index.html',

 'output\\https---books.toscrape.com-catalogue-grey-fifty-shades-4_592-index.html',

 'output\\https---books.toscrape.com-catalogue-harry-potter-and-the-chamber-of-secrets-harry-potter-2_325-index.html',

 'output\\https---books.toscrape.com-catalogue-harry-potter-and-the-half-blood-prince-harry-potter-6_326-index.html',

 'output\\https---books.toscrape.com-catalogue-harry-potter-and-the-order-of-the-phoenix-harry-potter-5_327-index.html',

 'output\\https---books.toscrape.com-catalogue-i-had-a-nice-time-and-other-lies-how-to-find-love-sht-like-that_814-index.html',

 'output\\https---books.toscrape.com-catalogue-keep-me-posted_594-index.html',

 'output\\https---books.toscrape.com-catalogue-meternity_478-index.html',

 'output\\https---books.toscrape.com-catalogue-page-33.html',

 'output\\https---books.toscrape.com-catalogue-page-35.html',

 'output\\https---books.toscrape.com-catalogue-paper-and-fire-the-great-library-2_339-index.html',

 'output\\https---books.toscrape.com-catalogue-soldier-talon-3_222-index.html',

 'output\\https---books.toscrape.com-catalogue-the-beast-black-dagger-brotherhood-14_342-index.html',

 'output\\https---books.toscrape.com-catalogue-the-rose-the-dagger-the-wrath-and-the-dawn-2_278-index.html',
 
 'output\\https---books.toscrape.com-catalogue-will-you-wont-you-want-me_644-index.html']
 ```

### Ejercicio extra de solución más abierta:

Haz ahora web scrapping para obtener un diccionario que contenga la siguiente información de cada libro que pertenezca a una lista de categorías: título, precio, disponibilidad, URL de la imagen, rating y URL del producto. Luego, almacena la información en un archivo CSV por categoría cuyo nombre tendrá el formato `categoria_numerolibros_books.csv` (p. ej. fiction_10_books.csv) y cuya primera línea contendrá los nombres de las columnas.

Ayudas:

- Ten en cuenta la estructura de las urls de los libros de una categoría. Por ejemplo, los de la categoría "fiction_10" tienen la siguiente URL: https://books.toscrape.com/catalogue/category/books/fiction_10/index.html
- Mira cómo se recorren las páginas dentro de una categoría: https://books.toscrape.com/catalogue/category/books/nonfiction_13/index.html
https://books.toscrape.com/catalogue/category/books/nonfiction_13/page-2.html
...
https://books.toscrape.com/catalogue/category/books/nonfiction_13/page-7.html -> provoca 404 Not Found
para eso puedes comprobar el atributo "status_code" de la respuesta de request.get(URL)


In [19]:
## BEGIN YOUR CODE

def get_rating_number(rating_class):
    """Convierte la clase de rating a número"""
    rating_map = {
        'One': 1,
        'Two': 2,
        'Three': 3,
        'Four': 4,
        'Five': 5
    }
    return rating_map.get(rating_class, 0)

def scrape_category(category_name, category_id, base_url="https://books.toscrape.com"):
    """
    Extrae información de todos los libros de una categoría
    
    Args:
        category_name: Nombre de la categoría (ej: 'fiction')
        category_id: ID de la categoría (ej: 10)
        base_url: URL base del sitio
    
    Returns:
        Lista de diccionarios con información de los libros
    """
    books_data = []
    page = 1
    
    while True:
        # Construir URL de la página
        if page == 1:
            url = f"{base_url}/catalogue/category/books/{category_name}_{category_id}/index.html"
        else:
            url = f"{base_url}/catalogue/category/books/{category_name}_{category_id}/page-{page}.html"
        
        # Hacer la petición
        response = requests.get(url)
        
        # Si la página no existe (404), salir del bucle
        if response.status_code == 404:
            print(f"  Página {page} no encontrada. Fin de la categoría.")
            break
        
        print(f"  Procesando página {page}...")
        
        # Parsear el contenido
        soup = Soup(response.text, 'html.parser')
        
        # Encontrar todos los libros en la página
        books = soup.find_all('article', class_='product_pod')
        
        for book in books:
            # Extraer título
            title = book.h3.a['title']
            
            # Extraer precio
            price = book.find('p', class_='price_color').text.strip()
            
            # Extraer disponibilidad
            availability = book.find('p', class_='instock availability').text.strip()
            
            # Extraer URL de la imagen
            img_url = book.find('img')['src']
            img_url = urljoin(url, img_url)  # Convertir a URL absoluta
            
            # Extraer rating
            rating_class = book.find('p', class_='star-rating')['class'][1]
            rating = get_rating_number(rating_class)
            
            # Extraer URL del producto
            product_url = book.h3.a['href']
            product_url = urljoin(url, product_url)  # Convertir a URL absoluta
            
            # Agregar información del libro
            books_data.append({
                'title': title,
                'price': price,
                'availability': availability,
                'image_url': img_url,
                'rating': rating,
                'product_url': product_url
            })
        
        page += 1
    
    return books_data

def save_to_csv(books_data, category_name, category_id):
    """
    Guarda la información de los libros en un archivo CSV
    
    Args:
        books_data: Lista de diccionarios con información de los libros
        category_name: Nombre de la categoría
        category_id: ID de la categoría
    """
    num_books = len(books_data)
    filename = f"{category_name}_{num_books}_books.csv"
    
    # Guardar en CSV
    with open(filename, 'w', newline='', encoding='utf-8') as f:
        if num_books > 0:
            # Usar las claves del primer diccionario como nombres de columnas
            fieldnames = books_data[0].keys()
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            
            # Escribir la cabecera
            writer.writeheader()
            
            # Escribir los datos
            writer.writerows(books_data)
    
    print(f"Archivo '{filename}' creado con {num_books} libros.\n")

# Lista de categorías a procesar (nombre, id)
categories = [
    ('fiction', 10),
    ('nonfiction', 13),
    ('mystery', 3),
    ('fantasy', 19),
    ('romance', 8)
]

# Procesar cada categoría
for category_name, category_id in categories:
    print(f"Procesando categoría: {category_name}_{category_id}")
    
    # Extraer información de los libros
    books_data = scrape_category(category_name, category_id)
    
    # Guardar en CSV
    save_to_csv(books_data, category_name, category_id)

print("¡Proceso completado!")

## END YOUR CODE

Procesando categoría: fiction_10
  Procesando página 1...
  Procesando página 2...
  Procesando página 3...
  Procesando página 4...
  Página 5 no encontrada. Fin de la categoría.
Archivo 'fiction_65_books.csv' creado con 65 libros.

Procesando categoría: nonfiction_13
  Procesando página 1...
  Procesando página 2...
  Procesando página 3...
  Procesando página 4...
  Procesando página 5...
  Procesando página 6...
  Página 7 no encontrada. Fin de la categoría.
Archivo 'nonfiction_110_books.csv' creado con 110 libros.

Procesando categoría: mystery_3
  Procesando página 1...
  Procesando página 2...
  Página 3 no encontrada. Fin de la categoría.
Archivo 'mystery_32_books.csv' creado con 32 libros.

Procesando categoría: fantasy_19
  Procesando página 1...
  Procesando página 2...
  Procesando página 3...
  Página 4 no encontrada. Fin de la categoría.
Archivo 'fantasy_48_books.csv' creado con 48 libros.

Procesando categoría: romance_8
  Procesando página 1...
  Procesando página 2...
